# This notebook creates the predictions for the test_numpy files.

_Author: Bálint Pataki_

In [ ]:
modelName = 'slightly_modified_model'

In [ ]:
#%env CUDA_VISIBLE_DEVICES=

In [ ]:
import pandas as pd
from deepLense import *
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
test_ids = !ls ../data/deepLenseData/test_numpy/
test_ids = [i[:-4] for i in test_ids]
#test_ids = np.random.choice(np.array(test_ids), replace=False, size=1000)

In [ ]:
model = keras.models.load_model('weights/slightly_modified_model/slightly_modified_model_04_0.000324.hdf5')

In [ ]:
true_list = []
omega_preds = []
sigma_preds = []
file_list = []

data = []
k = 0
for i in pb(test_ids):
    k += 1
    data.append(np.load('/mnt/deepLenseData/test_numpy/' + i + '.npy').reshape(256, 256, 1))
    file_list.append(i[:-4])
    omega = float(i.split('_')[0][2:])
    sigma = float(i.split('_')[1][2:])
    true_list.append([omega, sigma])
    
    if((k%32) == 0):        
        preds = model.predict_on_batch(np.array(data))        
        omega_preds = omega_preds + list(preds[:,0])
        sigma_preds = sigma_preds + list(preds[:,1])
        data = []
if(len(data)>0):        
    preds = model.predict_on_batch(np.array(data))        
    omega_preds = omega_preds + list(preds[:,0])
    sigma_preds = sigma_preds + list(preds[:,1])
    data = []
    
predDF = pd.DataFrame({'true_omega':np.array(true_list)[:,0],
              'true_sigma':np.array(true_list)[:,1],
              'pred_omega':omega_preds,
              'pred_sigma':sigma_preds,
              'fileName':file_list})
predDF = predDF.groupby('fileName').mean().reset_index() # average for big FITS file (16 smaller numpy array)

In [ ]:
predDF.to_csv('preds/' + modelName + '_preds.csv', index=False)